In [108]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20

In [109]:
!pip install chromadb==0.5.0
!pip install faiss-cpu

In [110]:
import openai
import numpy as np
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import faiss
import warnings
import os
warnings.filterwarnings("ignore")


In [111]:
openai.api_key = " "

In [112]:
os.environ['OPENAI_API_KEY'] = ' '

In [113]:
system_prompt = """
Role:
You are an intelligent data assistant specialized in inventory management for a manufacturing company. Your role is to create, manage, and provide insights on inventory datasets to enhance the company’s operational efficiency.

Instruct:
Your task is to help generate and maintain comprehensive inventory datasets, answer questions about the inventory, and provide recommendations or visual insights (e.g., charts and summaries). When requested, create or edit records, calculate inventory values, and identify patterns such as low stock levels or upcoming restock needs.

Context:
The company deals with a variety of products, including hardware, tools, safety gear, and supplies, stored in different warehouse locations. The dataset comprises details such as product IDs, names, SKUs, quantities, suppliers, unit prices, and restock dates. Accurate inventory management helps avoid shortages, streamline reorders, and maintain healthy stock levels.

Constraints:
Ensure data is kept accurate, up-to-date, and formatted consistently.
Use clear, understandable language when explaining data or providing instructions.
Maintain data integrity by avoiding duplication and ensuring proper calculations for fields like Total Inventory Value.
Only suggest feasible actions for inventory management that align with typical industry practices.
Do not provide or assume sensitive data such as real customer contact details unless provided explicitly in hypothetical scenarios.
Examples:
Task: Add a new product to the inventory.

Response: "To add a new product, please provide the following: Product ID, Product Name, Category, SKU, Quantity, Unit Price, Supplier Name, and Location. Once provided, I will create the entry."
Task: Calculate total value for a specific product.

Response: "The total value for Widget A (Product ID 001) is calculated as 500 units * $5 per unit, which equals $2,500."
Task: Identify low stock items.

Response: "Items such as Safety Goggles (Product ID 004) have a quantity of 75, which is close to the reorder level of 20. A restock may be needed soon."
Task: List products by restock date.

Response: "The next products to be restocked include Adhesive Tape on 2024-11-05, Paint Can (1L) on 2024-11-01, and Electrical Wire on 2024-11-04."
By following these guidelines, you will effectively support the company's inventory management needs and enhance productivity.

Closing Note: Ensure each interaction ends with an offer for further assistance, such as: "Is there anything else I can assist you with?".
"""

In [114]:
struct = [{'role' : 'system', 'content' : system_prompt}]

In [115]:
dataframed = pd.read_csv('https://raw.githubusercontent.com/vaniebermudez/ai_first_bootcamp/refs/heads/main/Day4/inventory_products_dataset.csv')
dataframed

,Product ID,Product Name,Category,SKU,Description,Supplier Name,Quantity in Stock,Reorder Level,Reorder Quantity,Unit Price,Total Inventory Value,Location,Date of Last Restock,Lead Time,Expiration Date,Batch Number,Condition,Barcode
0,1,Widget A,Hardware,WID123,High-quality metal widget,ABC Supplies,500,100,200,$5,$2500,Warehouse 1,11/01/2024,7 days,NaN,BATCH001,New,QR001
1,2,Bolt Pack,Fasteners,BOLT456,Set of 50 steel bolts,XYZ Fasteners,250,50,100,$2,$500,Warehouse 2,10/30/2024,5 days,NaN,BATCH002,New,QR002
2,3,Adhesive Tape,Supplies,TAP789,Industrial adhesive tape,TapeTech Inc.,150,30,50,$3,$450,Warehouse 3,11/05/2024,3 days,NaN,BATCH003,New,QR003
3,4,Safety Goggles,Safety Gear,SAF001,Protective eyewear,SafeWare Ltd.,75,20,50,$8,$600,Warehouse 1,11/02/2024,10 days,NaN,BATCH004,New,QR004
4,5,Welding Rods,Welding,WEL123,High-strength welding rods,WeldCo Supplies,120,40,80,$10,$1200,Warehouse 2,11/01/2024,15 days,NaN,BATCH005,New,QR005
5,6,Drill Bit Set,Tools,DRI456,10-piece drill bit set,ToolMasters Inc.,200,50,100,$15,$3000,Warehouse 3,10/29/2024,8 days,NaN,BATCH006,New,QR006
6,7,Electrical Wire,Electrical,ELE789,100m spool of copper wire,Electra Corp.,400,100,200,$20,$8000,Warehouse 1,11/04/2024,12 days,NaN,BATCH007,New,QR007
7,8,Hammer,Tools,HAM001,Heavy-duty hammer,HammerWorks Co.,90,20,50,$7,$630,Warehouse 3,11/03/2024,5 days,NaN,BATCH008,New,QR008
8,9,Paint Can (1L),Paints,PAI123,1-liter can of paint,ColorWorld,300,75,150,$12,$3600,Warehouse 2,11/01/2024,10 days,04/01/2025,BATCH009,New,QR009
9,10,Sandpaper Pack,Supplies,SAN456,Assorted sandpaper sheets,SmoothTech,500,100,250,$1,$500,Warehouse 1,11/06/2024,4 days,NaN,BATCH010,New,QR010


In [116]:
dataframed['combined'] = dataframed.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
dataframed

,Product ID,Product Name,Category,SKU,Description,Supplier Name,Quantity in Stock,Reorder Level,Reorder Quantity,Unit Price,Total Inventory Value,Location,Date of Last Restock,Lead Time,Expiration Date,Batch Number,Condition,Barcode,combined
0,1,Widget A,Hardware,WID123,High-quality metal widget,ABC Supplies,500,100,200,$5,$2500,Warehouse 1,11/01/2024,7 days,NaN,BATCH001,New,QR001,1 Widget A Hardware WID123 High-quality metal ...
1,2,Bolt Pack,Fasteners,BOLT456,Set of 50 steel bolts,XYZ Fasteners,250,50,100,$2,$500,Warehouse 2,10/30/2024,5 days,NaN,BATCH002,New,QR002,2 Bolt Pack Fasteners BOLT456 Set of 50 steel ...
2,3,Adhesive Tape,Supplies,TAP789,Industrial adhesive tape,TapeTech Inc.,150,30,50,$3,$450,Warehouse 3,11/05/2024,3 days,NaN,BATCH003,New,QR003,3 Adhesive Tape Supplies TAP789 Industrial adh...
3,4,Safety Goggles,Safety Gear,SAF001,Protective eyewear,SafeWare Ltd.,75,20,50,$8,$600,Warehouse 1,11/02/2024,10 days,NaN,BATCH004,New,QR004,4 Safety Goggles Safety Gear SAF001 Protective...
4,5,Welding Rods,Welding,WEL123,High-strength welding rods,WeldCo Supplies,120,40,80,$10,$1200,Warehouse 2,11/01/2024,15 days,NaN,BATCH005,New,QR005,5 Welding Rods Welding WEL123 High-strength we...
5,6,Drill Bit Set,Tools,DRI456,10-piece drill bit set,ToolMasters Inc.,200,50,100,$15,$3000,Warehouse 3,10/29/2024,8 days,NaN,BATCH006,New,QR006,6 Drill Bit Set Tools DRI456 10-piece drill bi...
6,7,Electrical Wire,Electrical,ELE789,100m spool of copper wire,Electra Corp.,400,100,200,$20,$8000,Warehouse 1,11/04/2024,12 days,NaN,BATCH007,New,QR007,7 Electrical Wire Electrical ELE789 100m spool...
7,8,Hammer,Tools,HAM001,Heavy-duty hammer,HammerWorks Co.,90,20,50,$7,$630,Warehouse 3,11/03/2024,5 days,NaN,BATCH008,New,QR008,8 Hammer Tools HAM001 Heavy-duty hammer Hammer...
8,9,Paint Can (1L),Paints,PAI123,1-liter can of paint,ColorWorld,300,75,150,$12,$3600,Warehouse 2,11/01/2024,10 days,04/01/2025,BATCH009,New,QR009,9 Paint Can (1L) Paints PAI123 1-liter can of ...
9,10,Sandpaper Pack,Supplies,SAN456,Assorted sandpaper sheets,SmoothTech,500,100,250,$1,$500,Warehouse 1,11/06/2024,4 days,NaN,BATCH010,New,QR010,10 Sandpaper Pack Supplies SAN456 Assorted san...


In [117]:
documents = dataframed['combined'].tolist()

In [118]:
embeddings = [get_embedding(doc, engine='text-embedding-3-small') for doc in documents]

In [119]:
embedding_dim = len(embeddings[0])

In [120]:
embedding_np = np.array(embeddings).astype('float32')

In [121]:
index = faiss.IndexFlatL2(embedding_dim)

In [122]:
index.add(embedding_np)

In [133]:
user_message = 'Can you identify which products in our inventory are below their reorder level and provide the reorder quantity for each of those products? Also, let me know the total value of the restock needed for those items.?'

In [134]:
query_embedding = get_embedding(user_message, engine='text-embedding-3-small')
query_embedding_np = np.array([query_embedding]).astype('float32')

In [135]:
_, indices = index.search(query_embedding_np, 10)

In [136]:
retrieved_docs = [documents[i] for i in indices[0]]

In [137]:
context = ' '.join(retrieved_docs)

In [138]:
structured_prompt = f'Context: \n{context} \n\n Query: {user_message} \n\n Response:'

In [139]:
print(structured_prompt)

Context: 
9 Paint Can (1L) Paints PAI123 1-liter can of paint ColorWorld 300 75 150 $12 $3600 Warehouse 2 11/01/2024 10 days 04/01/2025 BATCH009 New QR009 14 Steel Pipes (1m) Hardware PIP123 1-meter steel pipe MetalWorks Co. 250 60 100 $25 $6250 Warehouse 2 10/31/2024 10 days nan BATCH014 New QR014 13 Wrench Set Tools WRE789 8-piece wrench set FixIt Tools 70 20 50 $18 $1260 Warehouse 1 11/04/2024 14 days nan BATCH013 New QR013 10 Sandpaper Pack Supplies SAN456 Assorted sandpaper sheets SmoothTech 500 100 250 $1 $500 Warehouse 1 11/06/2024 4 days nan BATCH010 New QR010 1 Widget A Hardware WID123 High-quality metal widget ABC Supplies 500 100 200 $5 $2500 Warehouse 1 11/01/2024 7 days nan BATCH001 New QR001 16 Mask (N95) Safety Gear MAS789 N95 protective mask SafeBreath Corp. 300 80 150 $2 $600 Warehouse 1 11/05/2024 3 days 02/01/2025 BATCH016 New QR016 18 PVC Pipes (2m) Hardware PVC456 2-meter PVC pipe PipeFlow Ltd. 80 20 40 $15 $1200 Warehouse 3 10/28/2024 8 days nan BATCH018 New QR018

In [140]:
chat =  openai.ChatCompletion.create(model = "gpt-4o-mini", messages = struct + [{"role": "user", "content" : structured_prompt}], temperature=0.5, max_tokens=1500, top_p=1, frequency_penalty=0, presence_penalty=0)
struct.append({"role": "user", "content": user_message})
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})

In [141]:
print(response)

Based on the provided inventory data, here are the products that are below their reorder levels along with their respective reorder quantities and the total value of restock needed for each item:

1. **Paint Can (1L)**
   - **Current Quantity:** 75
   - **Reorder Level:** 150
   - **Reorder Quantity:** 150 - 75 = 75
   - **Unit Price:** $12
   - **Total Value for Restock:** 75 * $12 = $900

2. **Wrench Set**
   - **Current Quantity:** 20
   - **Reorder Level:** 50
   - **Reorder Quantity:** 50 - 20 = 30
   - **Unit Price:** $18
   - **Total Value for Restock:** 30 * $18 = $540

3. **Mask (N95)**
   - **Current Quantity:** 80
   - **Reorder Level:** 150
   - **Reorder Quantity:** 150 - 80 = 70
   - **Unit Price:** $2
   - **Total Value for Restock:** 70 * $2 = $140

4. **PVC Pipes (2m)**
   - **Current Quantity:** 20
   - **Reorder Level:** 40
   - **Reorder Quantity:** 40 - 20 = 20
   - **Unit Price:** $15
   - **Total Value for Restock:** 20 * $15 = $300

5. **Welding Rods**
   - **Cu